# Multisig Wallet tutorial and playground with JavaScript & web3.js
Tested on Energy Web Foundation's Tobalaba test network.


Please make 3 test accounts with some test tokens for experimentation

In [ ]:
const Web3 = require('web3');
const fs = require("fs");
const eventutils = require("./utils.js");

// Deployed factory address on Tobalaba
const factoryWithDLAddress = "0x7d4ae47c29790f22f157982d84445fa8e2c6e178"

const factoryWithDLAbi = JSON.parse(fs.readFileSync("contracts/MultiSigWalletWithDailyLimitFactory.json"))["abi"]
const walletWithDLAbi = JSON.parse(fs.readFileSync("contracts/MultiSigWalletWithDailyLimit.json"))["abi"]

const ADDRESS_EMPTY = '0x0000000000000000000000000000000000000000'

In [ ]:
const provider = new Web3.providers.HttpProvider("http://localhost:8545")
const w3 = new Web3(provider)

// you should have 3 accounts to play around
console.log(w3.eth.accounts)
w3.eth.defaultAccount = w3.eth.accounts[0]
//w3.personal.unlockAccount(w3.eth.defaultAccount, "passwordhere")

### 0. Let's create the factory instance

In [ ]:
FactoryWithDailyLimit = w3.eth.contract(factoryWithDLAbi)
WalletWithDailyLimit = w3.eth.contract(walletWithDLAbi)

factoryWithDailyLimit = FactoryWithDailyLimit.at(factoryWithDLAddress)
console.log("---")

### 1. Let's create a multisig wallet
You need:
 - the owner account addresses
 - how many confirmations are needed to perform transactions
 - daily limit -> the amount that can be withrdrawn per day without the confirmation of others

These setting can be later changed

2 ways to create the wallet:
 1. using the deployed wallet factory (simpler)
 2. compiling and deploying the Wallet contract yourself
 
The first method is demonstrated here

In [ ]:
var address1 = w3.eth.accounts[0]
var address2 = w3.eth.accounts[1]
var address3 = w3.eth.accounts[2]

var requiredConfirmations = 2
var dailyLimit = 0

var txHash = factoryWithDailyLimit.create([address1, address2, address3], requiredConfirmations, dailyLimit, {"from": address1, "gas": 5000000})

The factory emits a ```ContractInstantiation(address sender, address instantiation)``` event in case of a newly created wallet. We can get the wallet's address by accessing it.
You can either read the event logs and parse the data from the transaction receipt, or set a filter for the event and scan for it. The first method is shown. Then the wallet contract is then instantiated using the address and its ABI.

In [ ]:
var tx = w3.eth.getTransactionReceipt(txHash)
var logs = eventutils.logParser(tx.logs, factoryWithDLAbi, 'ContractInstantiation')
var myWalletAddress = logs[0].args.instantiation

var myWallet = w3.eth.contract(walletWithDLAbi).at(myWalletAddress)

console.log("Your deployed wallet address is: " + myWalletAddress)
console.log("Owners are: " + myWallet.getOwners.call())
console.log("Daily withdraw limit w/o confirmations: " + myWallet.dailyLimit.call() + " wei")
console.log("Allowed withdraw for today w/o confirmations: " + myWallet.calcMaxWithdraw.call()+ " wei")
console.log("Required confirmations: " + myWallet.required.call())

### 2. Let's send some play tokens to the wallet
E.g. 2 ethers for playing around

In [ ]:
w3.eth.sendTransaction({"from": w3.eth.accounts[0],
                        "to": myWalletAddress,
                        "value": w3.toWei(2, "ether")})

### 3. Let's try to withdraw some money back to account 1
 1. We submit a transaction invoking the ```submitTransaction(address destination, uint value, bytes data)```. ```value``` is where the transferrable 'money' goes in wei. More about the ```data``` field below in secton 5.
 2. We need to get the transaction ID: the wallet emits a ```Submission(uint indexed transactionId)``` event in case of  successful submission. We can read it out from the receipt.

In [ ]:
var txHash2 = myWallet.submitTransaction.sendTransaction(address1, w3.toWei(1, "ether"), 0,{"from": address1})

In [ ]:
var tx2 = w3.eth.getTransactionReceipt(txHash2)
var logs2 = logParser(tx2.logs, walletWithDLAbi, 'Submission')
var transactionId = logs2[0].args.transactionId

console.log("Transaction id: "+ transactionId)

#### 3.1 Let's check out the state of our submission

In [ ]:
console.log("Transaction count: " + myWallet.transactionCount.call())
console.log("Our transaction [destination/value/data/executed]: " + myWallet.transactions.call(transactionId))
console.log("Confirmed by " + address1 +": " + myWallet.confirmations.call(transactionId, address1))
console.log("Confirmed by " + address2 +": " + myWallet.confirmations.call(transactionId, address2))
console.log("Confirmed by " + address3 +": " + myWallet.confirmations.call(transactionId, address3))

#### 3.2 You can check that as long as the submission is not confirmed by at least another owner, you cannot send it

In [ ]:
myWallet.executeTransaction.sendTransaction(transactionId, {"from": address1, "gas": 800000})

In [ ]:
console.log("Our transaction [destination/value/data/executed]: " + myWallet.transactions.call(transactionId))

### 4. Confirm the transaction with other owners
 - ```confirmTransaction(uint transactionId)```
 - needs the transaction ID, and the sender needs to be the owner who confirms
 - a confirmed transaction can be executed by ``` executeTransaction(transactionId)```
 - ``` executeTransaction(transactionId)``` is automatically triggered if the number of confirmations reach the required with this last ```confirmTransaction``` and all conditions are met


In [ ]:
myWallet.confirmTransaction.sendTransaction(transactionId, {"from": address2, "gas": 800000})

In [ ]:
console.log("Our transaction [destination/value/data/executed]: " + myWallet.transactions.call(transactionId))

You can see that the transaction status is executed and the money appears on you destination account.

### 5. Change daily limit 
You cannot calll the wallet's changeDailyLimit function directly, it needs to be a confirmed transaction. You can change the daily limit, owners and required confirmation as well, with the consent of the owners.

#### How to invoke Smart Contract methods using your Multisig Wallet

If you look at the function signature ```submitTransaction(address destination, uint value, bytes data)``` you notice a ```bytes data``` field. It is used to invoke functionalities of a contract and can be left empty or bytes(0) for regular value transfers. Remember that invoking SC functions is just a regular transaction containing the relevant calldata. Calldata is obtained by encoding the desired function's signature and its parameters, but fortunately web3 libraries already do the heavy lifting for you, so no need to do this manually. The recipient address needs to be the address of the Smart Contract whose method you want to invoke.

In this case we want to invoke the ```changeDailyLimit``` function of our wallet Smart Contract. We need the ABI to encode the calldata easily.

In [ ]:
var callData = myWallet.changeDailyLimit.getData(w3.toWei(1, "ether"))
var txHash3=myWallet.submitTransaction.sendTransaction(myWalletAddress, 0, callData, {"from": address1})

After this points everything goes as with any other transaction from our wallet

In [ ]:
var tx3 = w3.eth.getTransactionReceipt(txHash3)
var logs3 = logParser(tx3.logs, walletWithDLAbi, 'Submission')
var transactionId = logs3[0].args.transactionId

console.log("Transaction id: "+ transactionId)

We confirm it with another account as well, which triggers the transaction

In [ ]:
myWallet.confirmTransaction.sendTransaction(transactionId, {"from": address2, "gas": 800000})

In [ ]:
if(myWallet.transactions.call(transactionId)[3] == true) {
    console.log("Transaction " + transactionId + " is executed")
    console.log("Daily withdraw limit w/o confirmations: " + myWallet.dailyLimit.call() + " wei")
} else {
    console.log("Transaction " + transactionId + "is not executed")
}

You should see that the daily limit has changed

### 6. Withdraw some ether w/o confirmation
##### Now that the daily limit is changed, it is time to test it. Calldata is 0.

In [ ]:
var txHash4 = myWallet.submitTransaction.sendTransaction(address1, w3.toWei(0.7, "ether"), 0, {"from": address1})

In [ ]:
var tx4 = w3.eth.getTransactionReceipt(txHash4)
var logs4 = logParser(tx4.logs, walletWithDLAbi, 'Submission')
var transactionId = logs4[0].args.transactionId

console.log("Transaction id: "+ transactionId)

In [ ]:
console.log("Transaction count: " + myWallet.transactionCount.call())
console.log("Our transaction [destination/value/data/executed]: " + myWallet.transactions.call(transactionId))
console.log("Confirmed by " + address1 +": " + myWallet.confirmations.call(transactionId, address1))
console.log("Confirmed by " + address2 +": " + myWallet.confirmations.call(transactionId, address2))
console.log("Confirmed by " + address3 +": " + myWallet.confirmations.call(transactionId, address3))

##### You can see that the transaction is executed, but only 1 account confirmed it. Let's check the remaining daily quota.
The daily quota is calculated for "today" by comparing the current time to a unix timestamp called ```lastDay```. If the curent moment is past ```lastDay + 24 hours``` then the daily quota resets. The lastDay timestamp is initially zero and is first set in the contract when we try to make a withdrawal.

In [ ]:
console.log("Daily withdraw limit w/o confirmations: " + myWallet.dailyLimit.call() + " wei")
console.log("Allowed withdraw for today w/o confirmations: " + myWallet.calcMaxWithdraw.call()+ " wei")
var lastDay = myWallet.lastDay.call()
console.log("Last day: " + lastDay + " -> " + new Date(lastDay*1000))

### 7. Let's remove an owner
Steps are very similar to changing the daily limit. You can manage ownership with```removeOwner(address owner)``` and```replaceOwner(address owner, address newOwner)``` methods, and change the confirmations needed with ```changeRequirement(uint _required)```.

I remove owner n3 in this example

In [ ]:
callData = myWallet.removeOwner.getData(address3)
var txHash5 = myWallet.submitTransaction.sendTransaction(myWalletAddress, 0, callData, {"from": address1})

In [ ]:
var tx5 = w3.eth.getTransactionReceipt(txHash5)
var logs5 = logParser(tx5.logs, walletWithDLAbi, 'Submission')
var transactionId = logs5[0].args.transactionId

console.log("Transaction id: "+ transactionId)

In [ ]:
myWallet.confirmTransaction.sendTransaction(transactionId, {"from": address2, "gas": 800000})

In [ ]:
if(myWallet.transactions.call(transactionId)[3] == true) {
    console.log("Transaction " + transactionId + " is executed")
    console.log("Owners are: " + myWallet.getOwners.call())
} else {
    console.log("Transaction " + transactionId + "is not executed")
}

##### You should see the chosen owner disappeared from the list

### 8. Your experiments here
Feel free to play around with your Multisig wallet